#  Energy Consumption Modeling for UAVs Based on Flight Data and Mission Parameters :
## 1. Data analysis and feature extraction


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


import plotly.express as px


In [3]:
df = pd.read_csv(r"C:\Users\raman\Desktop\Kanchan_Material\01_AirTransportAndLogistics\Course_Material\Semester_03\Research Task\12683453\flights.csv", delimiter=",", encoding="utf-8", header=0)
df.iloc[0:20, 0:28] 
#df.shape
print(df.columns.tolist())

['flight', 'time', 'wind_speed', 'wind_angle', 'battery_voltage', 'battery_current', 'position_x', 'position_y', 'position_z', 'orientation_x', 'orientation_y', 'orientation_z', 'orientation_w', 'velocity_x', 'velocity_y', 'velocity_z', 'angular_x', 'angular_y', 'angular_z', 'linear_acceleration_x', 'linear_acceleration_y', 'linear_acceleration_z', 'speed', 'payload', 'altitude', 'date', 'time_day', 'route']


C:\Users\raman\AppData\Local\Temp\ipykernel_15244\2303164508.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\raman\Desktop\Kanchan_Material\01_AirTransportAndLogistics\Course_Material\Semester_03\Research Task\12683453\flights.csv", delimiter=",", encoding="utf-8", header=0)


### Find Altitude, Vertical speed, Horizontal speed.

In [4]:
# Compute altitude relative to takeoff (up = positive)
df["altitude_measured"] = df["position_z"] - df.groupby("flight")["position_z"].transform("first")

# Compute vertical speed (m/s) from altitude_measured
df["vertical_speed"] = (
    df.groupby("flight")["altitude_measured"]
      .diff() / df.groupby("flight")["time"].diff()
)
print(df[df["flight"] == 1][["time", "altitude_measured", "vertical_speed"]].head(10))

df["vertical_speed_smooth"] = (
    df.groupby("flight")["vertical_speed"]
      .transform(lambda x: x.rolling(window=5, min_periods=1).mean())
)

# Compute horizontal speed (m/s)
df["horizontal_speed"] = np.sqrt(df["velocity_x"]**2 + df["velocity_y"]**2)

df["horizontal_speed_smooth"] = (
    df.groupby("flight")["horizontal_speed"]
      .transform(lambda x: x.rolling(window=5, min_periods=1).mean())
)

print(df[df["flight"] == 1][["time", "horizontal_speed"]].head(10))








   time  altitude_measured  vertical_speed
0  0.00           0.000000             NaN
1  0.20          -0.000346       -0.001728
2  0.30           0.000679        0.010252
3  0.50           0.002246        0.007835
4  0.60           0.003776        0.015297
5  0.81           0.003535       -0.001147
6  0.90           0.004929        0.015487
7  1.10           0.001680       -0.016246
8  1.20           0.000032       -0.016484
9  1.40           0.001752        0.008605
   time  horizontal_speed
0  0.00          0.014013
1  0.20          0.015496
2  0.30          0.016614
3  0.50          0.009389
4  0.60          0.010433
5  0.81          0.009317
6  0.90          0.009128
7  1.10          0.006518
8  1.20          0.004427
9  1.40          0.005595


### Find total acceralation, average power, peak power, Energy per flight

In [5]:
df[["linear_acceleration_x", "linear_acceleration_y", "linear_acceleration_z"]].head(50)


df["accel_total"] = np.sqrt(
    df["linear_acceleration_x"]**2 +
    df["linear_acceleration_y"]**2 +
    df["linear_acceleration_z"]**2
)
df[["battery_current", "battery_voltage"]].head(50)




# 1. Instantaneous electrical power (make positive when discharging)
df["power_W"] = -(df["battery_voltage"] * df["battery_current"])

# 2. Average and peak power
avg_power = df.groupby("flight")["power_W"].mean()
peak_power = df.groupby("flight")["power_W"].max()

energy_per_flight_Wh = (
    df.loc[:, ["flight", "time", "power_W"]]
      .groupby("flight")[["time", "power_W"]]         # explicitly exclude 'flight'
      .apply(lambda g: np.trapezoid(g["power_W"], g["time"]) / 3600.0)
      .rename("energy_Wh")
)




### Find different phase of flight using  altitude, horizontal and vertical speed. 

In [13]:

# Thresholds
vertical_threshold_up = 2.250    # m/s (positive means going up)
vertical_threshold_down = -0.7 # m/s (negative means going down)
altitude_threshold = 0.90       # m (to ensure actual height change)
hs_hover = 0.9                 # m/s, low horizontal speed
hs_cruise = 1.05                # m/s, high horizontal speed

# Start with a default phase
df['phase'] = 'other'

# --- Climb: positive vertical speed + above ground ---
climb_mask = (
    
    (df['altitude_measured'] > altitude_threshold)
)
df.loc[climb_mask, 'phase'] = 'climb'

# --- Descent: negative vertical speed + above ground ---
descent_mask = (
    (df['vertical_speed_smooth'] < vertical_threshold_down) &
    (df['altitude_measured'] > altitude_threshold)
)
df.loc[descent_mask, 'phase'] = 'descent'

# --- Hover: nearly no vertical motion, low horizontal speed ---
hover_mask = (
    (df['vertical_speed_smooth'].abs() < 0.7) &
    (df['horizontal_speed_smooth'] < hs_hover) &
    (df['altitude_measured'] > altitude_threshold)
)
df.loc[hover_mask, 'phase'] = 'hover'

# --- Cruise: nearly no vertical motion, high horizontal speed ---
cruise_mask = (
    (df['vertical_speed_smooth'] >= -0.7) &
    (df['vertical_speed_smooth'] < vertical_threshold_up) &
    (df['horizontal_speed_smooth'] > hs_cruise) &
    (df['altitude_measured'] > altitude_threshold)
)
df.loc[cruise_mask, 'phase'] = 'cruise'
#df.iloc[100:120, 0:35] 




### Visualize different phase of flight with Line graph

In [14]:


# ------------------ Select one flight ------------------
flight_id = 12
d = df[df["flight"] == flight_id].copy()

# ------------------ Define colors for phases ------------------
phase_colors = {
    "climb":   "rgba(0,255,0,0.25)",      # light green
    "descent": "rgba(255,0,0,0.25)",      # light red
    "hover":   "rgba(255,165,0,0.25)",    # light orange
    "cruise":  "rgba(0,0,255,0.25)",      # light blue
    "other":   "rgba(150,150,150,0.15)"   # light gray
}

# ------------------ Build customdata for hover ------------------
# customdata[:, 0] = altitude
# customdata[:, 1] = vertical speed
# customdata[:, 2] = horizontal speed
# customdata[:, 3] = phase
customdata = np.stack((
    d["altitude_measured"],
    d["vertical_speed_smooth"],
    d["horizontal_speed_smooth"],
    d["phase"]
), axis=-1)

# ------------------ Compute altitude range ------------------
y_min = d["altitude_measured"].min() - 1
y_max = d["altitude_measured"].max() + 1

fig = go.Figure()

# ------------------ 1) Add Altitude (left axis) ------------------
fig.add_trace(go.Scatter(
    x=d["time"],
    y=d["altitude_measured"],
    mode="lines",
    line=dict(width=2, color="blue"),
    name="Altitude (m)",
    customdata=customdata,
    hovertemplate=(
        "Time: %{x:.2f} s<br>"
        "Altitude: %{customdata[0]:.2f} m<br>"
        "Vertical speed: %{customdata[1]:.2f} m/s<br>"
        "Horizontal speed: %{customdata[2]:.2f} m/s<br>"
        "Phase: %{customdata[3]}<extra></extra>"
    )
))

# ------------------ 2) Add Vertical Speed (right axis) ------------------
fig.add_trace(go.Scatter(
    x=d["time"],
    y=d["vertical_speed_smooth"],
    mode="lines",
    line=dict(width=2, color="red"),
    name="Vertical speed (m/s)",
    yaxis="y2",
    customdata=customdata,
    hovertemplate=(
        "Time: %{x:.2f} s<br>"
        "Altitude: %{customdata[0]:.2f} m<br>"
        "Vertical speed: %{customdata[1]:.2f} m/s<br>"
        "Horizontal speed: %{customdata[2]:.2f} m/s<br>"
        "Phase: %{customdata[3]}<extra></extra>"
    )
))

# ------------------ 3) Add Phase Bands as Filled Traces ------------------
for phase, color in phase_colors.items():
    df_phase = d[d["phase"] == phase]
    if df_phase.empty:
        continue

    x_band = []
    y_band = []
    prev_idx = None
    start_t = None

    for idx, row in df_phase.iterrows():
        t = row["time"]
        if (prev_idx is None) or (idx != prev_idx + 1):
            if start_t is not None:
                # close previous rectangle
                x_band += [start_t, end_t, end_t, start_t, start_t, None]
                y_band += [y_min, y_min, y_max, y_max, y_min, None]
            start_t = t
        end_t = t
        prev_idx = idx

    # close last segment
    if start_t is not None:
        x_band += [start_t, end_t, end_t, start_t, start_t, None]
        y_band += [y_min, y_min, y_max, y_max, y_min, None]

    fig.add_trace(go.Scatter(
        x=x_band,
        y=y_band,
        mode="lines",
        fill="toself",
        fillcolor=color,
        line=dict(width=0),
        name=f"Phase: {phase}",
        hoverinfo="skip",  # don't show tooltip for background
        showlegend=True
    ))

# ------------------ 4) Layout ------------------
fig.update_layout(
    title=f"Flight {flight_id} – Altitude, Vertical Speed & Phases",
    xaxis=dict(title="Time (s)"),

    # Left Y-axis
    yaxis=dict(
        title=dict(text="Altitude (m)", font=dict(color="blue")),
        tickfont=dict(color="blue"),
        range=[y_min, y_max]
    ),

    # Right Y-axis
    yaxis2=dict(
        title=dict(text="Vertical speed (m/s)", font=dict(color="red")),
        tickfont=dict(color="red"),
        overlaying="y",
        side="right"
    ),

    template="plotly_white",
    legend=dict(x=1, y=1)
)
fig.update_layout(
    legend=dict(
        x=1.08,
        y=1,
        bgcolor="rgba(255,255,255,0.7)"
    ),
    margin=dict(r=120)
)


fig.show()
